#### Color Filtering

In [6]:
import cv2
import numpy as np

# def range of yellow color HSV color space
lower_yellow = np.array([20, 0, 0])
upper_yelllow = np.array([41,255, 255])

cap = cv2.VideoCapture(0)

while 1:
    ret, frame = cap.read()
    
    # get hsv colorspace image
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # gets mask of only color in the specified color Range
    mask = cv2.inRange(hsv_img, lower_yellow, upper_yelllow)
    
    # bitwise AND opp on mask and original frame and ret an image
    result = cv2.bitwise_and(frame, frame, mask = mask)
    
    cv2.imshow("mask", mask)
    cv2.imshow("Filtered Color only", result)
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()

***
#### Background Subtraction

In [3]:
### Guassian Mixture-Based Background2
import numpy as np
import cv2

cap = cv2.VideoCapture("D:/Computer_Vision_Recap/images/Walking.avi")

# init background subtractor
foreground_background = cv2.createBackgroundSubtractorKNN()

while 1:
    ret, frame = cap.read()
    
    if frame is None:
        break

    # apply background subtractor to get foreground mask
    foreground_mask = foreground_background.apply(frame)
    
    cv2.imshow("Guassian Mixture-Based Background", foreground_mask)
    if(cv2.waitKey(1) == 13):
        break
        
cap.release()
cv2.destroyAllWindows()

In [9]:
#### BackGround Subtraction KNN #####
import cv2
import numpy as np

cap = cv2.VideoCapture("D:/Computer_Vision_Recap/images/Walking.avi")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
foreground_background = cv2.createBackgroundSubtractorKNN()

while 1:
    ret, frame = cap.read()
    
    foreground_mask = foreground_background.apply(frame)
    foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_OPEN, kernel)
    
    cv2.imshow("Background sub KNN", foreground_mask)
    
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()

In [5]:
#### FOREGROUND SUBTRACTION ####
import cv2
import numpy as np


cap = cv2.VideoCapture(0)

ret, frame = cap.read()
avg = np.float32(frame)

while 1:
    ret, frame = cap.read()
    
    # 0.01 is weight of image, looks at movement in frame and accumulates
    # weights for things not moving
    cv2.accumulateWeighted(frame, avg, 0.01)
    
    # conversion to show to user
    background = cv2.convertScaleAbs(avg)
    
    cv2.imshow("Input", frame)
    cv2.imshow("Foreground subtraction", background)
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()

***
### Object Tracking

In [3]:
#### MeanShift Object Tracking #####

import numpy as np
import cv2

cap = cv2.VideoCapture(0)

ret, frame = cap.read()

# setsup default location of tracking window
y1, y2, x1, x2 = 200, 150, 500, 178
track_window = (x1, y1, x2, y2)

# crops ROI for tracking
roi = frame[y1:y1+y2, x1:x1+x2]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)


# creates mask within hsv color space range
lower_yellow = np.array([20, 0, 0])
upper_yelllow = np.array([43,255, 255])
mask = cv2.inRange(hsv_roi, lower_yellow, upper_yelllow)

# gets color hist of ROI which is needed to normalize values btw 0 - 255
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [50], [0, 50])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

"""
setsup termination criteria, i.e stop calc when centroid shift after ten
iteration and the centroid doesn't move by @ least 1 pixel
"""
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while 1:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    if ret:      # true when reading succesfull
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # GETS hist back propagation i.e prob of a pixel valu and
        # is needed by meanashift
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 50], 1)
        
        # apply meanshift to get new loc
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        x1, y1, x2, y2 = track_window
        img_2 = cv2.rectangle(frame, (x1, y1), (x1+x2, y1+y2), 255, 2)
        cv2.imshow("MeanShift Tracking", img_2)
        
        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
        
cv2.destroyAllWindows()
cap.release()
        
        


In [4]:
#### CamShift Object Tracking ###

import numpy as np
import cv2

cap = cv2.VideoCapture(0)

ret, frame = cap.read()

# setsup default location of tracking window
y1, y2, x1, x2 = 200, 150, 500, 178
track_window = (x1, y1, x2, y2)

# crops ROI for tracking
roi = frame[y1:y1+y2, x1:x1+x2]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)


# creates mask within hsv color space range
lower_yellow = np.array([20, 0, 0])
upper_yelllow = np.array([43,255, 255])
mask = cv2.inRange(hsv_roi, lower_yellow, upper_yelllow)

# gets color hist of ROI which is needed to normalize values btw 0 - 255
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [50], [0, 50])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

"""
setsup termination criteria, i.e stop calc when centroid shift after ten
iteration and the centroid doesn't move by @ least 1 pixel
"""
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while 1:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    if ret:      # true when reading succesfull
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # GETS hist back propagation i.e prob of a pixel value and
        # is needed by camshift
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 50], 1)
        
        # apply camShift to get new loc
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # draw new tracking window on frame and use PolyLines to rep adaptive Box
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img_2 = cv2.polylines(frame, [pts], True, 255, 2)
        cv2.imshow("CamShfit Object Tracking", img_2)
        
        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
        
cv2.destroyAllWindows()
cap.release()

***
#### Optical Flow Object Tracking

In [3]:
### Lucas Kanade OpticalFlow(only keypoints)###
import cv2
import numpy as np

cap = cv2.VideoCapture("D:/Computer_Vision_Recap/images/Walking.avi")

# sets params for shiTomasi corner detection
feature_params = dict(maxCorners = 100, qualityLevel = 0.3,
                     minDistance = 7, blockSize = 7)

# sets params for LK opticalFlow
lucas_kanade_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors for trailing image movement in frame
color = np.random.randint(0,255,(100,3))

# Take first frame and find init corners in it
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(prev_frame)

while(1):
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    new_corners, status, errors = cv2.calcOpticalFlowPyrLK(prev_gray, 
                                                           frame_gray, 
                                                           prev_corners, 
                                                           None, 
                                                           **lucas_kanade_params)

    # Select and store good points(new and old corners)
    good_new = new_corners[status==1]
    good_old = prev_corners[status==1]

    # Draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (a,b), (c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a,b), 5, color[i].tolist(),-1)
        
    img = cv2.add(frame,mask)

    # Show Optical Flow
    cv2.imshow('Optical Flow - Lucas-Kanade',img)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prev_corners = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

<ipython-input-3-c1174819d454>:46: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv2.line(mask, (a,b), (c,d), color[i].tolist(), 2)
<ipython-input-3-c1174819d454>:47: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame, (a,b), 5, color[i].tolist(),-1)


In [4]:
### Dense Optical Flow(Slower than LK Optical Flow) ###

import cv2
import numpy as np


cap = cv2.VideoCapture("D:/Computer_Vision_Recap/images/Walking.avi")

# Get first frame
ret, first_frame = cap.read()
previous_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(first_frame)
hsv[...,1] = 255

while True:
    
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    # Computes the dense optical flow using the Gunnar Farneback’s algorithm
    flow = cv2.calcOpticalFlowFarneback(previous_gray, next, 
                                        None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # use flow to calculate the magnitude (speed) and angle of motion
    # use these values to calculate the color to reflect speed and angle
    magnitude, angle = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = angle * (180 / (np.pi/2))
    hsv[...,2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    final = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    
    cv2.imshow('Dense Optical Flow', final)
    if cv2.waitKey(1) == 13: 
        break
    
    # Store current image as previous image
    previous_gray = next

cap.release()
cv2.destroyAllWindows()

